# 🎯 LangGraph Hello World

欢迎来到LangGraph世界！这个notebook将引导你创建第一个LangGraph应用。

## 📚 学习目标

通过这个教程，你将学习到：
- LangGraph的基本概念
- 如何创建简单的工作流
- StateGraph和MessageGraph的区别
- 如何运行和测试你的图

## ⏱️ 预计时间：15分钟

---

## 1. 🎨 什么是LangGraph？

LangGraph是一个强大的框架，用于构建**有状态的、多步骤的AI应用**。它让你能够：

- 🔗 **连接多个处理步骤**
- 💾 **在步骤间传递状态**
- 🔀 **基于条件进行路由**
- 🧠 **持久化对话状态**
- 🛠️ **集成外部工具**

可以把LangGraph想象成一个可视化的流水线，每个节点执行特定任务，边决定数据流向。

## 2. 🚀 环境设置

In [ ]:
# 安装必要的依赖（如果还没有安装）
# !pip install langgraph langchain langchain-core

# 导入必要的模块
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, MessageGraph
from typing import TypedDict, Annotated
import operator

print("✅ 所有模块导入成功！")
print(f"📦 LangGraph版本: {getattr(__import__('langgraph'), '__version__', 'unknown')}")

## 3. 📝 第一个StateGraph示例

StateGraph是LangGraph中最常用的图类型，它使用**状态字典**来在节点间传递数据。

In [ ]:
# 定义状态结构
class ChatState(TypedDict):
    """聊天状态定义"""
    messages: Annotated[list, operator.add]  # 消息列表
    user_name: str  # 用户名字
    step_count: int  # 处理步骤计数

print("📋 状态结构定义完成")
print("🔧 状态字段:")
print("  - messages: 消息列表 (会自动合并)")
print("  - user_name: 用户名字符串")
print("  - step_count: 整数计数器")

In [ ]:
# 定义聊天机器人节点函数
def chatbot_node(state: ChatState):
    """简单的聊天机器人处理函数"""
    
    # 获取当前状态
    messages = state.get("messages", [])
    step_count = state.get("step_count", 0) + 1
    user_name = state.get("user_name", "朋友")
    
    # 获取最后一条人类消息
    human_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
    
    if not human_messages:
        # 没有人类消息，发送欢迎消息
        response = f"你好{user_name}！我是你的AI助手。"
    else:
        last_message = human_messages[-1].content
        
        # 简单的回复逻辑
        if "你好" in last_message or "hello" in last_message.lower():
            response = f"你好{user_name}！很高兴认识你！"
        elif "名字" in last_message and user_name == "朋友":
            # 用户在介绍名字
            import re
            name_match = re.search(r'我叫|我是|叫我\s*([^\s，。！？]+)', last_message)
            if name_match:
                user_name = name_match.group(1)
                response = f"很高兴认识你，{user_name}！"
            else:
                response = "很高兴认识你！可以告诉我你的名字吗？"
        elif "功能" in last_message:
            response = f"{user_name}，LangGraph是一个强大的框架，可以构建复杂的AI工作流！"
        else:
            response = f"{user_name}，你说：'{last_message}'"
    
    # 返回更新后的状态
    return {
        "messages": [AIMessage(content=response)],
        "step_count": step_count,
        "user_name": user_name
    }

print("🤖 聊天机器人节点函数定义完成")
print("🔧 功能特点:")
print("  - 识别用户姓名")
print("  - 记住对话状态")
print("  - 个性化回复")
print("  - 步骤计数")

In [ ]:
# 创建StateGraph
graph = StateGraph(ChatState)

# 添加节点
graph.add_node("chatbot", chatbot_node)

# 设置入口点
graph.set_entry_point("chatbot")

# 设置出口点
graph.set_finish_point("chatbot")

# 编译图
compiled_graph = graph.compile()

print("✅ StateGraph创建完成！")
print("📊 图结构:")
print("  入口: chatbot")
print("  节点: chatbot")
print("  出口: chatbot")
print("\n🎯 这是一个简单的单节点图，适合基础对话")

## 4. 🎬 运行第一个LangGraph应用

In [ ]:
# 测试1: 初始问候
print("🎬 测试1: 初始问候")
print("=" * 40)

initial_state = {
    "messages": [],
    "user_name": "",
    "step_count": 0
}

result = await compiled_graph.ainvoke(initial_state)

print(f"输入: (无用户输入)")
print(f"用户名: {result['user_name']}")
print(f"步骤数: {result['step_count']}")
print(f"回复: {result['messages'][-1].content}")

In [ ]:
# 测试2: 用户介绍名字
print("🎬 测试2: 用户介绍名字")
print("=" * 40)

result2 = await compiled_graph.ainvoke({
    "messages": [HumanMessage(content="你好，我叫小明")],
    "user_name": "朋友",
    "step_count": 0
})

print(f"输入: 你好，我叫小明")
print(f"识别的用户名: {result2['user_name']}")
print(f"回复: {result2['messages'][-1].content}")

In [ ]:
# 测试3: 记住用户的个性化对话
print("🎬 测试3: 个性化对话")
print("=" * 40)

result3 = await compiled_graph.ainvoke({
    "messages": [HumanMessage(content="LangGraph有什么功能？")],
    "user_name": "小明",  # 之前识别的名字
    "step_count": 0
})

print(f"输入: LangGraph有什么功能？")
print(f"用户名: {result3['user_name']}")
print(f"回复: {result3['messages'][-1].content}")

## 5. 🔄 MessageGraph替代方案

MessageGraph是更简单的选择，专注于消息流而不需要复杂的状态管理。

In [ ]:
# 创建MessageGraph
message_graph = MessageGraph()

def simple_responder(messages):
    """简单的消息响应器"""
    last_message = messages[-1].content if messages else ""
    
    if "hello" in last_message.lower():
        return AIMessage(content="Hello there! 👋")
    elif "测试" in last_message:
        return AIMessage(content="测试成功！🎉")
    else:
        return AIMessage(content=f"收到消息: {last_message}")

# 设置图结构
message_graph.add_node("responder", simple_responder)
message_graph.set_entry_point("responder")
message_graph.set_finish_point("responder")

compiled_message_graph = message_graph.compile()

print("✅ MessageGraph创建完成！")
print("🔄 特点: 专注于消息流，更简单的状态管理")

In [ ]:
# 测试MessageGraph
print("🎬 测试MessageGraph")
print("=" * 40)

test_messages = [
    [HumanMessage(content="Hello World")],
    [HumanMessage(content="测试消息")],
    [HumanMessage(content="任意消息")]
]

for i, msgs in enumerate(test_messages, 1):
    result = await compiled_message_graph.ainvoke(msgs)
    print(f"测试{i}: {msgs[0].content}")
    print(f"回复: {result[-1].content}")
    print()

## 6. 🔧 可视化图结构

LangGraph提供了可视化工具来查看你的图结构。

In [ ]:
# 获取图的ASCII表示
print("📊 StateGraph结构:")
print(compiled_graph.get_graph().print_ascii())

print("\n📊 MessageGraph结构:")
print(compiled_message_graph.get_graph().print_ascii())

## 7. 🧪 互动练习

现在轮到你了！尝试修改代码来完成以下任务：

In [ ]:
# 练习1: 扩展聊天机器人功能
# 修改chatbot_node函数，添加对"时间"和"天气"的响应

def enhanced_chatbot_node(state: ChatState):
    """增强版聊天机器人"""
    
    # TODO: 在这里添加你的代码
    # 提示:
    # 1. 检查消息中是否包含"时间"
    # 2. 如果是，返回当前时间
    # 3. 检查消息中是否包含"天气"
    # 4. 如果是，返回天气信息（模拟）
    
    messages = state.get("messages", [])
    human_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
    
    if not human_messages:
        response = "你好！我是增强版AI助手。"
    else:
        last_message = human_messages[-1].content
        
        # 在这里添加你的逻辑
        if "时间" in last_message:
            import datetime
            current_time = datetime.datetime.now().strftime("%H:%M:%S")
            response = f"现在时间是 {current_time}"
        elif "天气" in last_message:
            response = "今天天气晴朗，25°C，适合学习编程！"
        else:
            response = f"你说: {last_message}"
    
    return {"messages": [AIMessage(content=response)]}

# 创建增强版图
enhanced_graph = StateGraph(ChatState)
enhanced_graph.add_node("enhanced_chatbot", enhanced_chatbot_node)
enhanced_graph.set_entry_point("enhanced_chatbot")
enhanced_graph.set_finish_point("enhanced_chatbot")

enhanced_compiled = enhanced_graph.compile()

print("✅ 增强版聊天机器人创建完成！")

# 测试增强功能
test_queries = ["现在几点了？", "今天天气如何？", "普通消息"]
for query in test_queries:
    result = await enhanced_compiled.ainvoke({
        "messages": [HumanMessage(content=query)]
    })
    print(f"输入: {query}")
    print(f"回复: {result['messages'][-1].content}")
    print()

## 8. 📊 状态管理深入

理解LangGraph的状态管理机制是掌握这个框架的关键。

In [ ]:
# 演示状态如何在不同节点间传递
class ProcessState(TypedDict):
    text: str
    processed_text: str
    step: str
    counter: int

def step1_processor(state: ProcessState):
    """步骤1: 初始处理"""
    return {
        "processed_text": state["text"].upper(),
        "step": "step1_completed",
        "counter": state.get("counter", 0) + 1
    }

def step2_processor(state: ProcessState):
    """步骤2: 进一步处理"""
    return {
        "processed_text": f"[处理完成] {state['processed_text']}",
        "step": "step2_completed",
        "counter": state["counter"] + 1
    }

# 创建多步骤图
process_graph = StateGraph(ProcessState)
process_graph.add_node("step1", step1_processor)
process_graph.add_node("step2", step2_processor)

process_graph.set_entry_point("step1")
process_graph.add_edge("step1", "step2")  # 连接步骤
process_graph.set_finish_point("step2")

process_compiled = process_graph.compile()

print("🔧 多步骤处理图创建完成")

# 运行处理
result = await process_compiled.ainvoke({
    "text": "hello langgraph",
    "processed_text": "",
    "step": "start",
    "counter": 0
})

print(f"\n📊 处理结果:")
print(f"原文: {result['text']}")
print(f"处理后: {result['processed_text']}")
print(f"步骤: {result['step']}")
print(f"计数: {result['counter']}")

## 9. 🎯 总结

恭喜！你已经完成了第一个LangGraph应用！让我们总结一下学到的知识：

### ✅ 你学会了：

1. **LangGraph基本概念**
   - StateGraph vs MessageGraph
   - 节点（Nodes）和边（Edges）
   - 状态管理

2. **创建工作流**
   - 定义状态结构（TypedDict）
   - 创建节点函数
   - 设置图结构

3. **运行和测试**
   - 使用`ainvoke()`执行图
   - 处理输入和输出
   - 调试和可视化

4. **高级特性**
   - 多步骤处理
   - 状态传递
   - 条件逻辑

### 🚀 下一步学习：

- 📖 **教程2**: 状态管理与持久化
- 🔄 **教程3**: 条件路由与决策
- 🛠️ **教程4**: 工具集成与API调用
- 🤖 **教程5**: 多代理系统

### 💡 最佳实践提醒：

- 🎯 保持节点函数简单和专注
- 📝 使用类型提示提高代码可读性
- 🔍 充分测试每个节点的功能
- 📚 逐步构建复杂的工作流

---

**🎉 恭喜完成Hello World教程！继续你的LangGraph学习之旅吧！**